In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
import pandas as pd

class VaersDescrReader:
    
    def __init__(self, dataDir):
        self.dataDir = dataDir        

    def readAllVaersDescrs(self):
        return self.readVaersDescrs(["2021", "2022"])
        
    def readVaersDescrs(self, years):
        return [self.readVaersDescr(year) for year in years]

    def readVaersDescr(self, year):
        folder = self.dataDir + "/" + year + "VAERSData/"
        return {
                    'VAERSDATA': self._readVAERSDATA(folder + year + "VAERSDATA.csv"),
                    'VAERSVAX': self._readVAERSVAX(folder + year + "VAERSVAX.csv")
               }

    def readNonDomesticVaersDescr(self):
        folder = self.dataDir + "/NonDomesticVAERSData/"
        return {
                    'VAERSDATA': self._readVAERSDATA(folder + "NonDomesticVAERSDATA.csv"),
                    'VAERSVAX': self._readVAERSVAX(folder + "NonDomesticVAERSVAX.csv")
               }

    def _readVAERSDATA(self, file):
        return self._read_csv(
            file = file,
            usecols = ['VAERS_ID', 'RECVDATE', 'DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT', 'SPLTTYPE'],
            parse_dates = ['RECVDATE'],
            date_parser = lambda dateStr: pd.to_datetime(dateStr, format = "%m/%d/%Y"))

    def _readVAERSVAX(self, file):
        return self._read_csv(
            file = file,
            usecols = ['VAERS_ID', 'VAX_DOSE_SERIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT'],
            dtype = {"VAX_DOSE_SERIES": "string"})

    def _read_csv(self, file, **kwargs):
        return pd.read_csv(
            file,
            index_col = 'VAERS_ID',
            encoding = 'latin1',
            low_memory = False,
            **kwargs)


In [ ]:
import pandas as pd

class VaersDescr2DataFrameConverter:

    @staticmethod
    def createDataFrameFromDescr(vaersDescr):
        return pd.merge(
                vaersDescr['VAERSDATA'],
                vaersDescr['VAERSVAX'],
                how = 'left',
                left_index = True,
                right_index = True,
                validate = 'one_to_many')

    @staticmethod
    def createDataFrameFromDescrs(vaersDescrs):
        dataFrames = [VaersDescr2DataFrameConverter.createDataFrameFromDescr(vaersDescr) for vaersDescr in vaersDescrs]
        return pd.concat(dataFrames)


In [ ]:
class DataFrameNormalizer:
    
    @staticmethod
    def normalize(dataFrame):
        DataFrameNormalizer.convertVAX_LOTColumnToUpperCase(dataFrame)
        DataFrameNormalizer._convertColumnsOfDataFrame_Y_to_1_else_0(
            dataFrame,
            ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'])

    @staticmethod
    def convertVAX_LOTColumnToUpperCase(dataFrame):
        dataFrame['VAX_LOT'] = dataFrame['VAX_LOT'].str.upper()

    @staticmethod
    def _convertColumnsOfDataFrame_Y_to_1_else_0(dataFrame, columns):
        for column in columns:
            DataFrameNormalizer._convertColumnOfDataFrame_Y_to_1_else_0(dataFrame, column)

    @staticmethod
    def _convertColumnOfDataFrame_Y_to_1_else_0(dataFrame, column):
        dataFrame[column] = DataFrameNormalizer._where(
            condition = dataFrame[column] == 'Y',
            trueValue = 1,
            falseValue = 0)

    @staticmethod
    def _where(condition, trueValue, falseValue):
        return np.where(condition, trueValue, falseValue)    
    

In [ ]:
import pandas as pd

class DataFrameFilter:
    
    def filterByCovid19(self, dataFrame):
        return dataFrame[self._isCovid19(dataFrame)]

    def filterByCountry(self, dataFrame, country, countryColumnName):
        return dataFrame[dataFrame[countryColumnName] == country]

    def filterBy(self, dataFrame, manufacturer = None, dose = None):
        return dataFrame[self._isManufacturer(dataFrame, manufacturer) & self._isDose(dataFrame, dose)]

    def _isCovid19(self, dataFrame):
        return dataFrame["VAX_TYPE"] == "COVID19"

    def _isManufacturer(self, dataFrame, manufacturer):
        return dataFrame["VAX_MANU"] == manufacturer if manufacturer is not None else True

    def _isDose(self, dataFrame, dose):
        return dataFrame["VAX_DOSE_SERIES"].str.contains(dose) if dose is not None else True


In [ ]:
class SummationTableFactory:

    @staticmethod
    def createSummationTable(
        groupBy,
        # FK-TODO: rename "ADRs" and "Total reports" to "Total Number of Adverse Reaction Reports" in all places
        columnNameMappingsDict = {
                "DIED_size": "ADRs",
                "DIED_sum": "DEATHS",
                "L_THREAT_sum": "LIFE THREATENING ILLNESSES",
                "DISABLE_sum": "DISABILITIES",
                'HOSPITAL_sum': 'HOSPITALISATIONS',
                'ER_VISIT_sum': 'EMERGENCY ROOM OR DOCTOR VISITS'
            }):

        summationTable = groupBy.agg({
                'DIED': ['sum', 'size'],
                'L_THREAT': 'sum',
                'DISABLE': 'sum',
                'HOSPITAL': 'sum',
                'ER_VISIT': 'sum'
            })
        SummationTableFactory._flattenColumns(summationTable)
        return summationTable.rename(columns = columnNameMappingsDict)

    @staticmethod
    def _flattenColumns(dataFrame):
        dataFrame.columns = ["_".join(a) for a in dataFrame.columns.to_flat_index()]


In [ ]:
import pandas as pd

class BatchCodeTableFactory:

    @staticmethod
    def createBatchCodeTable(dataFrame : pd.DataFrame, manufacturer, dose):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        dataFrame = DataFrameFilter().filterBy(dataFrame, manufacturer = manufacturer, dose = dose)
        return BatchCodeTableFactory._createSummationTableByVAX_LOT(dataFrame)[['ADRs', 'DEATHS', 'DISABILITIES', 'LIFE THREATENING ILLNESSES']]

    # create table from https://www.howbadismybatch.com/combined.html
    @staticmethod
    def createSevereEffectsBatchCodeTable(dataFrame : pd.DataFrame, dose):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        dataFrame = DataFrameFilter().filterBy(dataFrame, dose = dose)
        return BatchCodeTableFactory._createSummationTableByVAX_LOT(dataFrame)

    @staticmethod
    def _createSummationTableByVAX_LOT(dataFrame):
        batchCodeTable = SummationTableFactory.createSummationTable(dataFrame.groupby('VAX_LOT'))
        batchCodeTable = batchCodeTable[['ADRs', 'DEATHS', 'DISABILITIES', 'LIFE THREATENING ILLNESSES', 'HOSPITALISATIONS', 'EMERGENCY ROOM OR DOCTOR VISITS']]
        batchCodeTable = batchCodeTable.sort_values(by = 'ADRs', ascending = False)
        return BatchCodeTableFactory._addCompanyColumn(batchCodeTable, BatchCodeTableFactory._createCompanyByBatchCodeTable(dataFrame))

    @staticmethod
    def _addCompanyColumn(batchCodeTable, companyByBatchCodeTable):
        return pd.merge(
            batchCodeTable,
            companyByBatchCodeTable,
            how = 'left',
            left_index = True,
            right_index = True,
            validate = 'one_to_one')

    @staticmethod
    def _createCompanyByBatchCodeTable(dataFrame):
        return BatchCodeTableFactory._createManufacturerByBatchCodeTable(dataFrame).rename(columns = {"VAX_MANU": "COMPANY"})

    @staticmethod
    def _createManufacturerByBatchCodeTable(dataFrame):
        manufacturerByBatchCodeTable = dataFrame[['VAX_LOT', 'VAX_MANU']]
        manufacturerByBatchCodeTable = manufacturerByBatchCodeTable.drop_duplicates(subset = ['VAX_LOT'])
        return manufacturerByBatchCodeTable.set_index('VAX_LOT')


In [ ]:
class DoseTableFactory:
    
    @staticmethod
    def createDoseTable(dataFrame):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        return DoseTableFactory._createDoseTable(
            dataFrame.groupby(
                dataFrame['VAX_DOSE_SERIES'].rename('Dose')))

    @staticmethod
    def createDoseByMonthTable(dataFrame):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        return DoseTableFactory._createDoseTable(
            dataFrame.groupby(
                [
                    dataFrame['RECVDATE'].dt.year.rename('Year'),
                    dataFrame['RECVDATE'].dt.month.rename('Month'),
                    dataFrame['VAX_DOSE_SERIES'].rename('Dose')
                ]))

    # FK-TODO: rename _createDoseTable()
    @staticmethod
    def _createDoseTable(dataFrame):
        doseTable = SummationTableFactory.createSummationTable(
            dataFrame,
            columnNameMappingsDict = {
                "DIED_size": "Total reports",
                "DIED_sum": "Deaths",
                "L_THREAT_sum": "Life Threatening Illnesses",
                "DISABLE_sum": "Disabilities"
            })
        doseTable = doseTable[['Total reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses']]
        doseTable['Severe reports (%)'] = (doseTable['Deaths'] + doseTable['Disabilities'] + doseTable['Life Threatening Illnesses']) / doseTable['Total reports'] * 100
        return doseTable


In [ ]:
import pycountry

class CountryColumnAdder:
    
    @staticmethod
    def addCountryColumn(dataFrame, countryColumnName):
        dataFrame[countryColumnName] = dataFrame.apply(
            lambda row:
                CountryColumnAdder._getCountryNameOfSplttypeOrDefault(
                 splttype = row['SPLTTYPE'],
                 default = 'Unknown Country'),
            axis = 'columns')
        return dataFrame.astype({countryColumnName: "string"})

    @staticmethod
    def _getCountryNameOfSplttypeOrDefault(splttype, default):
        if not isinstance(splttype, str):
            return default
        
        country = pycountry.countries.get(alpha_2 = splttype[:2])
        return country.name if country is not None else default

In [ ]:
class InternationalLotTableFactory:
    
    @staticmethod
    def createInternationalLotTable(dataFrame):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        internationalLotTable = InternationalLotTableFactory._createInternationalLotTable(dataFrame)
        return internationalLotTable.sort_values(by = 'Severe reports (%)', ascending = False)

    @staticmethod
    def createBatchCodeTableByCountry(dataFrame : pd.DataFrame, country):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        batchCodeTable = InternationalLotTableFactory._createBatchCodeTableByCountry(dataFrame, country)
        return batchCodeTable.sort_values(by = 'Severe reports (%)', ascending = False)

    @staticmethod
    def _createInternationalLotTable(dataFrame):
        countryColumnName = 'Country'
        dataFrame = CountryColumnAdder.addCountryColumn(dataFrame, countryColumnName = countryColumnName)
        return DoseTableFactory._createDoseTable(dataFrame.groupby(dataFrame[countryColumnName]))

    @staticmethod
    def _createBatchCodeTableByCountry(dataFrame : pd.DataFrame, country):
        countryColumnName = 'Country'
        dataFrame = CountryColumnAdder.addCountryColumn(dataFrame, countryColumnName = countryColumnName)
        dataFrame = DataFrameFilter().filterByCountry(dataFrame, country = country, countryColumnName = countryColumnName)
        return DoseTableFactory._createDoseTable(dataFrame.groupby('VAX_LOT'))


In [ ]:
import unittest

In [ ]:
class TestHelper:

    @staticmethod
    def createDataFrame(index, columns, data, dtypes = {}):
        return pd.DataFrame(index = index, columns = columns, data = data).astype(dtypes)


In [ ]:
from pandas.testing import assert_frame_equal

class DataFrameNormalizerTest(unittest.TestCase):

    def test_convertVAX_LOTColumnToUpperCase(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['VAX_LOT'],
            data = [  ['037K20A'],
                      ['025l20A'],
                      ['025L20A']],
            index = [
                "0916600",
                "0916601",
                "1996874"])
            
        # When
        DataFrameNormalizer.convertVAX_LOTColumnToUpperCase(dataFrame)
        
        # Then
        dataFrameExpected = TestHelper.createDataFrame(
            columns = ['VAX_LOT'],
            data = [  ['037K20A'],
                      ['025L20A'],
                      ['025L20A']],
            index = [
                "0916600",
                "0916601",
                "1996874"])
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)


In [ ]:
from pandas.testing import assert_frame_equal

class DataFrameFilterTest(unittest.TestCase):

    def test_filterByCovid19_filterBy(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [1,      0,          0],
                                  [0,      0,          1]],
                        index = [
                            "0916600",
                            "0916601"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',  '037K20A', '1'],
                                  ['COVID19',  'MODERNA',  '025L20A', '1']],
                        index = [
                            "0916600",
                            "0916601"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                },
                {
                        'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [0,       0,         0],
                                  [0,       0,         1]],
                            index = [
                            "1996873",
                            "1996874"]),
                        'VAERSVAX': TestHelper.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['HPV9',     'MERCK & CO. INC.', 'R017624', 'UNK'],
                                      ['COVID19',  'MODERNA',          '025L20A', '1']],
                            index = [
                                "1996873",
                                "1996874"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    }
            ])
        dataFrameFilter = DataFrameFilter()
            
        # When
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)
        dataFrame = dataFrameFilter.filterBy(dataFrame, manufacturer = "MODERNA", dose = '1')
        
        # Then
        dataFrameExpected = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,       0,         0,         'COVID19',  'MODERNA',  '037K20A', '1'],
                      [0,       0,         1,         'COVID19',  'MODERNA',  '025L20A', '1'],
                      [0,       0,         1,         'COVID19',  'MODERNA',  '025L20A', '1']],
            index = [
                "0916600",
                "0916601",
                "1996874"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_filterByDose(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                        data = [  [1,      1,          0,         1,          1],
                                  [0,      0,          1,         0,          1]],
                        index = [
                            "0916600",
                            "0916601"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',         '037K20A', '1'],
                                  ['COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
                        index = [
                            "0916600",
                            "0916601"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        dataFrameFilter = DataFrameFilter()
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)

        # When
        dataFrame = dataFrameFilter.filterBy(dataFrame, dose = '1')
        
        # Then
        dataFrameExpected = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT', 'VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,      1,          0,         1,          1,          'COVID19',  'MODERNA',         '037K20A', '1'],
                      [0,      0,          1,         0,          1,          'COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
            index = [
                "0916600",
                "0916601"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_filterByFirstDose(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [1,      0,          0]],
                        index = [
                            "1048786"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',  '016M20A', '2'],
                                  ['COVID19',  'MODERNA',  '030L20A', '1']],
                        index = [
                            "1048786",
                            "1048786"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        dataFrameFilter = DataFrameFilter()
            
        # When
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)
        dataFrame = dataFrameFilter.filterBy(dataFrame, manufacturer = "MODERNA", dose = '1')
        
        # Then
        dataFrameExpected = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '030L20A', '1']],
            index = [
                "1048786"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_filterBySecondDose(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [1,      0,          0]],
                        index = [
                            "1048786"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',  '016M20A',  '2'],
                                  ['COVID19',  'MODERNA',  '030L20A',  '1']],
                        index = [
                            "1048786",
                            "1048786"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        dataFrameFilter = DataFrameFilter()

        # When
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)
        dataFrame = dataFrameFilter.filterBy(dataFrame, manufacturer = "MODERNA", dose = '2')
        
        # Then
        dataFrameExpected = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '016M20A', '2']],
            index = [
                "1048786"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)


In [ ]:
from pandas.testing import assert_frame_equal

class BatchCodeTableFactoryTest(unittest.TestCase):

    def testcreateSummationTable(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                        data = [  [1,      1,          0,         1,          1],
                                  [0,      0,          1,         0,          1]],
                        index = [
                            "0916600",
                            "0916601"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',         '037K20A', '1'],
                                  ['COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
                        index = [
                            "0916600",
                            "0916601"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])

        # When
        batchCodeTable = BatchCodeTableFactory.createSevereEffectsBatchCodeTable(dataFrame, '1')

        # Then
        batchCodeTableExpected = pd.DataFrame(
            data = {
                'ADRs': [1, 1],
                'DEATHS': [0, 1],
                'DISABILITIES': [1, 0],
                'LIFE THREATENING ILLNESSES': [0, 1],
                'HOSPITALISATIONS': [0, 1],
                'EMERGENCY ROOM OR DOCTOR VISITS': [1, 1],
                'COMPANY': ['PFIZER\BIONTECH', 'MODERNA']
            },
            index = pd.Index(['025L20A', '037K20A'], name = 'VAX_LOT'))
        assert_frame_equal(batchCodeTable, batchCodeTableExpected, check_dtype = False)

    def test_createBatchCodeTable2(self):
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
               {
                   'VAERSDATA': TestHelper.createDataFrame(
                       columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                       data = [  [1,      0,          0,         0,          0],
                                 [0,      0,          1,         0,          0]],
                       index = [
                           "0916600",
                           "0916601"]),
                   'VAERSVAX': TestHelper.createDataFrame(
                       columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                       data = [  ['COVID19',  'MODERNA',  '037K20A', '1'],
                                 ['COVID19',  'MODERNA',  '025L20A', '1']],
                       index = [
                           "0916600",
                           "0916601"],
                       dtypes = {'VAX_DOSE_SERIES': "string"})
               },
               {
                    'VAERSDATA': TestHelper.createDataFrame(
                       columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                       data = [  [0,      0,          0,         0,          0],
                                 [0,      0,          1,         0,          0]],
                       index = [
                           "1996873",
                           "1996874"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['HPV9',     'MERCK & CO. INC.', 'R017624', 'UNK'],
                                  ['COVID19',  'MODERNA',          '025L20A', '1']],
                        index = [
                            "1996873",
                            "1996874"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        self._test_createBatchCodeTable(dataFrame, "MODERNA", '1')

    def test_createBatchCodeTable(self):
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            VaersDescrReader(dataDir = "test/VAERS").readAllVaersDescrs())
        DataFrameNormalizer.normalize(dataFrame)
        self._test_createBatchCodeTable(dataFrame, "MODERNA", '1')

    def _test_createBatchCodeTable(self, dataFrame, manufacturer, dose):
        # When
        batchCodeTable = BatchCodeTableFactory.createBatchCodeTable(dataFrame, manufacturer, dose)

        # Then
        batchCodeTableExpected = pd.DataFrame(
            data = {
                'ADRs': [2, 1],
                'DEATHS': [0, 1],
                'DISABILITIES': [2, 0],
                'LIFE THREATENING ILLNESSES': [0, 0]
            },
            index = pd.Index(['025L20A', '037K20A'], name = 'VAX_LOT'))
        assert_frame_equal(batchCodeTable, batchCodeTableExpected, check_dtype = False)


In [ ]:
from pandas.testing import assert_frame_equal

class DoseTableFactoryTest(unittest.TestCase):

    def test_createDoseTable(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'HOSPITAL', 'ER_VISIT'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '016M20A', '2',               0,          0],
                      [1,      0,          0,         'COVID19',  'MODERNA',  '030L20A', '1',               0,          0],
                      [1,      1,          1,         'COVID19',  'MODERNA',  '030L20B', '1',               0,          0]],
            index = [
                "1048786",
                "1048786",
                "4711"],
                dtypes = {'VAX_DOSE_SERIES': "string"})
        
        # When
        doseTable = DoseTableFactory.createDoseTable(dataFrame)

        # Then
        assert_frame_equal(
            doseTable,
            pd.DataFrame(
                data = {
                    'Total reports':              [2,                   1],
                    'Deaths':                     [2,                   1],
                    'Disabilities':               [1,                   0],
                    'Life Threatening Illnesses': [1,                   0],
                    'Severe reports (%)':         [(2 + 1 + 1)/2 * 100, (1 + 0 + 0)/1 * 100]
                },
                index = pd.Index(['1', '2'], dtype = "string", name = 'Dose')))
                
    def test_createDoseByMonthTable(self):
        # Given
        parseDate = lambda dateStr: pd.to_datetime(dateStr, format = "%m/%d/%Y")
        dataFrame = TestHelper.createDataFrame(
            columns = ['RECVDATE',             'DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'HOSPITAL', 'ER_VISIT'],
            data = [  [parseDate('01/01/2021'), 1,     0,          0,         'COVID19',  'MODERNA',  '016M20A', '2',               0,          0],
                      [parseDate('01/01/2021'), 1,     0,          0,         'COVID19',  'MODERNA',  '030L20A', '1',               0,          0],
                      [parseDate('01/01/2021'), 1,     1,          1,         'COVID19',  'MODERNA',  '030L20B', '1',               0,          0]],
            index = [
                "1048786",
                "1048786",
                "4711"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        
        # When
        doseByMonthTable = DoseTableFactory.createDoseByMonthTable(dataFrame)

        # Then
        assert_frame_equal(
            doseByMonthTable,
            pd.DataFrame(
                data = {
                    'Total reports':              [2,                   1],
                    'Deaths':                     [2,                   1],
                    'Disabilities':               [1,                   0],
                    'Life Threatening Illnesses': [1,                   0],
                    'Severe reports (%)':         [(2 + 1 + 1)/2 * 100, (1 + 0 + 0)/1 * 100]
                },
                index = pd.MultiIndex.from_tuples(
                    [
                        (2021,  1, '1'),
                        (2021,  1, '2'),
                    ],
                names = ('Year', 'Month', 'Dose'))),
                check_index_type = False)


In [ ]:
from pandas.testing import assert_frame_equal

class InternationalLotTableFactoryTest(unittest.TestCase):

    def test_createInternationalLotTable(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'SPLTTYPE',                  'HOSPITAL', 'ER_VISIT'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '016M20A', '2',               'GBPFIZER INC2020486806',    0,          0],
                      [1,      0,          0,         'COVID19',  'MODERNA',  '030L20A', '1',               'FRMODERNATX, INC.MOD20224', 0,          0],
                      [1,      1,          1,         'COVID19',  'MODERNA',  '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0],
                      [0,      0,          0,         'COVID19',  'MODERNA',  '030L20B', '1',               'dummy'],
                      [0,      0,          0,         'COVID19',  'MODERNA',  '030L20B', '1',               123]],
            index = [
                "1048786",
                "1048786",
                "4711",
                "0815",
                "0816"])
        
        # When
        internationalLotTable = InternationalLotTableFactory.createInternationalLotTable(dataFrame)

        # Then
        assert_frame_equal(
            internationalLotTable,
            TestHelper.createDataFrame(
                columns = ['Total reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses', 'Severe reports (%)'],
                data = [  [2,               2,        1,              1,                            (2 + 1 + 1) / 2 * 100],
                          [1,               1,        0,              0,                            (1 + 0 + 0) / 1 * 100],
                          [2,               0,        0,              0,                            (0 + 0 + 0) / 2 * 100]],
                index = pd.Index(
                    [
                        'France',
                        'United Kingdom',
                        'Unknown Country'
                    ],
                    dtype = "string",
                    name = 'Country')))

    def test_createBatchCodeTableByCountry(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'SPLTTYPE',                  'HOSPITAL', 'ER_VISIT'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '016M20A', '2',               'GBPFIZER INC2020486806',    0,          0],
                      [1,      0,          0,         'COVID19',  'MODERNA',  '030L20A', '1',               'FRMODERNATX, INC.MOD20224', 0,          0],
                      [1,      1,          1,         'COVID19',  'MODERNA',  '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0],
                      [0,      1,          1,         'COVID19',  'MODERNA',  '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0]],
            index = [
                "1048786",
                "1048786",
                "4711",
                "0815"])
        
        # When
        batchCodeTable = InternationalLotTableFactory.createBatchCodeTableByCountry(dataFrame, 'France')

        # Then
        assert_frame_equal(
            batchCodeTable,
            TestHelper.createDataFrame(
                columns = ['Total reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses', 'Severe reports (%)'],
                data = [  [2,               1,        2,              2,                            (1 + 2 + 2) / 2 * 100],
                          [1,               1,        0,              0,                            (1 + 0 + 0) / 1 * 100]],
                index = pd.Index(
                    [
                        '030L20B',
                        '030L20A'
                    ],
                    name = 'VAX_LOT')),
            check_dtype = False)


In [ ]:
unittest.main(argv = [''], verbosity = 2, exit = False)

In [ ]:
import os

def ensurePath(file):
    directory = os.path.dirname(file)
    if not os.path.exists(directory):
        os.makedirs(directory)

def saveDataFrameAsExcelFile(dataFrame, file):
    ensurePath(file)
    dataFrame.to_excel(file)


In [ ]:
def saveBatchCodeTable(manufacturer, excelFile):
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    DataFrameNormalizer.normalize(dataFrame)
    batchCodeTable = BatchCodeTableFactory.createBatchCodeTable(dataFrame, manufacturer = manufacturer, dose = '1')
    display(batchCodeTable)
    saveDataFrameAsExcelFile(batchCodeTable, excelFile)

### Moderna batch codes

In [ ]:
# https://www.howbadismybatch.com/moderna.html
saveBatchCodeTable("MODERNA", "results/batchCodes/moderna.xlsx")

### Pfizer batch codes

In [ ]:
# https://www.howbadismybatch.com/pfizer.html
saveBatchCodeTable("PFIZER\BIONTECH", "results/batchCodes/pfizer.xlsx")

### Janssen batch codes 

In [ ]:
# https://www.howbadismybatch.com/janssen.html
saveBatchCodeTable("JANSSEN", "results/batchCodes/janssen.xlsx")

### Short-list of 2000 batches having severe effects

In [ ]:
def saveSevereEffectsBatchCodeTable(excelFile):
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    DataFrameNormalizer.normalize(dataFrame)
    severeEffectsBatchCodeTable = BatchCodeTableFactory.createSevereEffectsBatchCodeTable(dataFrame, dose = '1')
    display(severeEffectsBatchCodeTable)
    saveDataFrameAsExcelFile(severeEffectsBatchCodeTable, excelFile)

In [ ]:
saveSevereEffectsBatchCodeTable('results/severeEffects.xlsx')

### Variation in Effect of First and Second Doses

In [ ]:
# https://www.howbadismybatch.com/firstsecond.html

def createDoseTable():
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    DataFrameNormalizer.normalize(dataFrame)
    return DoseTableFactory.createDoseTable(dataFrame)

def createDoseByMonthTable():
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    DataFrameNormalizer.normalize(dataFrame)
    return DoseTableFactory.createDoseByMonthTable(dataFrame)

In [ ]:
createDoseTable()

In [ ]:
doseByMonthTable = createDoseByMonthTable()
saveDataFrameAsExcelFile(doseByMonthTable, 'results/firstsecond/doseByMonthTable.xlsx')
doseByMonthTable

### International Deadly Lots

In [ ]:
# https://www.howbadismybatch.com/international.html

def getNonDomesticVaers():
    vaersDescr = VaersDescrReader(dataDir = 'VAERS').readNonDomesticVaersDescr()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescr(vaersDescr)
    DataFrameNormalizer.normalize(dataFrame)
    return dataFrame

nonDomesticVaers = getNonDomesticVaers()

In [ ]:
internationalLotTable = InternationalLotTableFactory.createInternationalLotTable(nonDomesticVaers)

In [ ]:
# FK-TODO: make filter on 'Total reports' a parameter in getInternationalLotTable() 
internationalLotTable = internationalLotTable[internationalLotTable['Total reports'] > 50]
saveDataFrameAsExcelFile(internationalLotTable, 'results/international/International_Deadly_Lots.xlsx')
internationalLotTable

In [ ]:
for country in ['United Kingdom', 'France', 'Germany', 'Japan', 'Italy', 'Austria', 'Netherlands', 'Spain', 'Belgium', 'Sweden', 'Portugal', 'Australia']:
    batchCodeTable = InternationalLotTableFactory.createBatchCodeTableByCountry(nonDomesticVaers, country)
    batchCodeTable = batchCodeTable[batchCodeTable['Total reports'] > 50]
    saveDataFrameAsExcelFile(batchCodeTable, 'results/international/' + country + '.xlsx')
    display(country + ":", batchCodeTable)